In [41]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain.agents import initialize_agent, load_tools
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
import pandas as pd
from datasets import load_dataset
from openai import OpenAI
from rouge import Rouge
from pprint import pprint

In [2]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs that can be easily swapped for a different model.
OPENAI_MODEL = "gpt-4-turbo"

# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
transcript = pd.read_json('../Resources/vertopal.com_Discuss Collaborative Testing prior to release.json', orient='index')
transcript.head(5)

0
﻿Transcript\nApril 29, 2025, 2:02PM\n\nZain Mas... NaN

## Bot to be used only for transcribing from Audio > Text (using Whisper)
Remove comments when transcribing Auido file into test to use the Whispher bot

Add/remove 'audio' from variable 'transcribed_audio' for non-audio files

In [11]:
'''#audio transcription bot
from openai import OpenAI

client = OpenAI()
audio_file = open("Discuss RCM _ CPQ Data Usecases-20250411_081350-Meeting Recording.mp3", 'rb')

transcribed_audio = client.audio.transcriptions.create(
    file=audio_file,
    model='whisper-1',
    response_format='text'
)
transcribed_audio = '"' + transcribed_audio + '"'
print(transcribed_audio)'''

'#audio transcription bot\nfrom openai import OpenAI\n\nclient = OpenAI()\naudio_file = open("Discuss RCM _ CPQ Data Usecases-20250411_081350-Meeting Recording.mp3", \'rb\')\n\ntranscribed_audio = client.audio.transcriptions.create(\n    file=audio_file,\n    model=\'whisper-1\',\n    response_format=\'text\'\n)\ntranscribed_audio = \'"\' + transcribed_audio + \'"\'\nprint(transcribed_audio)'

In [31]:
# Calling it the same variable for simplicity later append '_audio' if using audio

transcribed_mtg = str(transcript[0])
len(transcribed_mtg)

56998

In [32]:
# Splitting transcription if max token limit is possibly exceeded
max_len = 37000

if len(transcribed_mtg) >= max_len:

    transcribed_mtg_trimmed_list = [transcribed_mtg[i:i + max_len] for i in range(0, len(transcribed_mtg), max_len)]

transcribed_mtg_trimmed_list

["\ufeffTranscript\\nApril 29, 2025, 2:02PM\\n\\nZain Master started transcription\\n\\nZain Master   0:03\\nEverybody, the purpose of this meeting is to talk about and understand and formulate plans on how to make sure that we can increase the confidence of our.\\nReleases going out to customers, right? As we understand most of our products are impacted by a level of customizations that involve very specific customer setups and implementations.\\nSo while the product is able to perform a level of testing, there needs to be that next level of testing that is performed in specific to that customer's implementation and that specific environment. Many times as we've seen, we've gotten a release out there only to have it.\\nImpacted either by product score functionality that needs to be compatible with a implementation, or the implementation side of the House.\\nThat's incompatible with product and then that gets us into this wonderful spiral of trying to continually address.\\nThe issue a

# Summarizer Bot that will summarize uploaded transcripts *(text/audio)* with key takeaway and learning

In [45]:
# Create Summary bot
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0)

# Prompt the bot to summarize meeting
prompt = ChatPromptTemplate.from_messages([("system", "Write a detailed summary of the meeting transciption that was uploaded:\\n\\n{text}")])

# Instantiate summarize chain with 'Stuff' as chain_type
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

# Create Document objects from the filtered transcripts
documents = transcribed_mtg_trimmed_list

docs = [Document(page_content=doc) for doc in documents]


# Batching the docs if they're too long
def batch_documents(docs, batch_size=1):
    for i in range(0, len(docs), batch_size):
        yield docs[i:i + batch_size]

batch_summaries = []
for batch in batch_documents(docs):
    combined_text = '\n\n'.join([doc.page_content for doc in batch])
    summary = chain.run(input_documents=batch)
    batch_summaries.append(summary)

# Summarize all summaries
final_input_docs = [Document(page_content=summary) for summary in batch_summaries]
final_summary = chain.run(input_documents=final_input_docs)

print(final_summary)

The meeting was centered around addressing the issue of increasing confidence in product releases to customers, particularly due to the high level of customizations in customer setups. The participants discussed the necessity for additional testing beyond standard product testing to guarantee compatibility with customer implementations and environments. They highlighted past challenges related to compatibility issues and stressed the importance of cross-collaboration between the solution center and core product teams.

Various action items were proposed, such as releasing updates in waves to detect issues early, increasing testing participation from the solution center, and securing QA resources in advance for testing. The meeting also touched upon managing communication, DevOps, and infrastructure during the release process.

The participants emphasized the need for a formalized process for testing, resource bookings, and communication throughout the release cycle. They stressed defin

## Lets evaluate Summarizer

In [52]:
# Lets evaluate the summarize using ROUGE (Recall-oriented understudy for Gisting Evaluations), which compares the output from the LLM against Human written summaries (Which I happend to have at hand as these were my meetings)
# ROUGE scores (ROUGE-1,ROUGE-2, ROUGE-L) provide a quantitative measure of the summary's quality.

def evaluate_summary(reference_summary, generated_summary):
    rouge = Rouge()
    scores = rouge.get_scores(generated_summary, reference_summary)[0]
    return scores

reference = """Core Product Team (Engineering and Product Management) met with members from the Solution Center, responsible for developing and managing customizations 
on customer's CPQ implementations to improve confidence in GA releases and customer adoption of the release. The focus of this meeting was to outline collaboration between 
Core Product and Solution Center to test the release candidate code prior to making it generally available for release. A process was defined on how Core Product teams can 
book and secure resources well in advance of the release from Solution Center to ensure testing of the code in customer environments. Core Product also agreed to demo'ing features in 
the release with members of the Solution Center to keep them in the loop. Product has also requested Solution Center engineers to save the output of outbound APIs before they hit customer intergrations/
applications to help with troubleshooting escalated issues. Multiple other topic discussions yet to be discussed (API performance testing, customer application security access, and test automation)
were to be discussed in a follow-up meeting to be scheduled by product management."""


scores = evaluate_summary(reference, final_summary)
print(scores)


{'rouge-1': {'r': 0.411214953271028, 'p': 0.23655913978494625, 'f': 0.3003412922918147}, 'rouge-2': {'r': 0.07692307692307693, 'p': 0.03858520900321544, 'f': 0.05139185850583975}, 'rouge-l': {'r': 0.35514018691588783, 'p': 0.20430107526881722, 'f': 0.25938566089249737}}


**NOTE:** Either I suck at writing meeting notes or the model sucks at evaluation. For when I read the summary  of the meeting via the Summarize we built, I am fairly certain that its capturing all of the main points. 

## Action Items Bot that will sift through the provided transcript and extract key action items for organizer to review

,Common Tasks Types,Average Task duration in minutes,Tasks weights
0,Send Email,10,0.25
1,Create Power point on known topic,120,2.00
2,Create Power point on new unfamiliar topic,240,3.00
3,Schedule Meeting,10,0.25
4,Work on small project,1200,5.00


In [59]:
# Prompt the bot to identify action items from the meeting transcript and try to
prompt2 = """You are an intelligent assistant tasked with extracting all unique calls to action from the following text. Return each call to action seperately with a newline character. For example:

-Call to Action 1
-Call to Action 2
-Call to Action 3
...

Here is the text:
"""
client = OpenAI()

# Instantiate chain
chain3 = client.chat.completions.create(model=OPENAI_MODEL, messages=[{'role': 'system', 'content': prompt2},
                                                                      {'role': 'user', 'content': transcribed_mtg}]
                                                                      )

calls = chain3.choices[0].message.content
print(calls)

-Generate and store output for integrations prior to every release to compare.
-Include the input from solution center into core development.
-Identify a set of customers that are either the most complex.
-Include SC resources in demo meetings to keep a loop of the customization work.
-Start API performance testing as part of the scope of the test.


In [65]:
# Create a dataframe for Bot to reference and assign task duration / weight to each action item
capacity = pd.read_csv('/Users/zainmaster/Desktop/Homeworks/Project_3_LLMs/Meeting-Bot/Resources/CommonTasks.csv')
capacity.head()

,Common Tasks Types,Average Task duration in minutes,Tasks weights
0,Send Email,10,0.25
1,Create Power point on known topic,120,2.00
2,Create Power point on new unfamiliar topic,240,3.00
3,Schedule Meeting,10,0.25
4,Work on small project,1200,5.00


In [ ]:
# Bot to assign a value (hours / weights) to the action items identified from the meeting transcription
prompt4 = """You are a smart and intelligent project manager that understand how much time it could take on average to complete a action item. 
Based on all of the action items / calls from the uploaded meeting transcript, you will reference a table of common tasks and assign to each task a approximate time value in minutes.
If a task is hard for you to assign a time to complete, list the task followed by 'Unable to estimate'
For every task to do, list the task followed by the duration on the same continuous line.
Your output should be a list of every task from the list, and their average time to complete.
Ensure each task entry and their duration to complete are one continuous line.
An Example Output List:

Task 1: (# Mins)
Task 2: (# Mins)
Name 3: (# Mins)
Name X: (Unable to estimate).
"""

# Instantiate chain
chain3 = client.chat.completions.create(
    model='gpt-4-turbo',
    messages=[{'role': 'system', 'content': prompt4},
              {'role': 'system', 'content': f"calls: {str(calls)}\n Meeting: {transcribed_audio}\n Call to action list: {meeting_results_action_list}"}]
)

names_calls = chain3.choices[0].message.content
name_calls = names_calls.strip('\n \n')
print(names_calls)

In [54]:
# Adding it all together, meeting summary and action items from the bots above
meeting_results = {'summary': final_summary, 'action items': calls}

meeting_results['summary'] = 'Summary:\n\n' + meeting_results['summary'] + '\n'
meeting_results['action calls'] = ('Action Items:\n' + meeting_results['action calls']).replace('\n', '\n\n')

print(meeting_results['summary'])
print(meeting_results['action calls'])

Summary:

The meeting was centered around addressing the issue of increasing confidence in product releases to customers, particularly due to the high level of customizations in customer setups. The participants discussed the necessity for additional testing beyond standard product testing to guarantee compatibility with customer implementations and environments. They highlighted past challenges related to compatibility issues and stressed the importance of cross-collaboration between the solution center and core product teams.

Various action items were proposed, such as releasing updates in waves to detect issues early, increasing testing participation from the solution center, and securing QA resources in advance for testing. The meeting also touched upon managing communication, DevOps, and infrastructure during the release process.

The participants emphasized the need for a formalized process for testing, resource bookings, and communication throughout the release cycle. They stre

In [41]:
# NER bot
prompt3 = """You are a smart and intelligent reviewer of the transcribed_audio meeting notes thats capable of Named Entity Recognition (NER). You will extract and display the unique names of those that attended the meeting and those that were mentioned but didnt attend from the uploaded transcribed_audio file. Importantly, your output should have each name separate with a newline character. For example:

Attendees:
Name 1
Name 2
Name 3
...

Didnt Attend but may have actions assigned to them:
Name 1 
Name 2
Name 3
...

Here is the text:
"""

# Instantiate chain
chain3 = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[{'role': 'system', 'content': prompt3},
              {'role': 'user', 'content': transcribed_audio}]
)

names = chain3.choices[0].message.content
print(names)

Attendees:
Zain Master
Lukas Kawulok
Stephen Pack
Terry Stone
Radim Moravec
Petr Klen

Didnt Attend but mentioned:
Chuy
Nalin


In [46]:
# Removing the 'Attendees' and 'Didnt Attend.....' from the list to create a email list
names = names.strip('Attendees:').replace('Didnt Attend but mentioned:', '')

In [47]:

names_list = names.split('\n')
names_list

['',
 'Zain Master',
 'Lukas Kawulok',
 'Stephen Pack',
 'Terry Stone',
 'Radim Moravec',
 'Petr Klen',
 '',
 '',
 'Chuy',
 'Nal']

In [58]:
# Removing Empty entries in a list
email_list =[]

for s in names_list:
    if s:
        email_list.append(s)

print(email_list)

['Zain Master', 'Lukas Kawulok', 'Stephen Pack', 'Terry Stone', 'Radim Moravec', 'Petr Klen', 'Chuy', 'Nal']


This is where we would need to have a user facing interface to ask the meeting organizer to verify names and the emails of the attendees that should be receiving the summary notes and action items from our application.

In [62]:
# dummy emails
members_emails = [
    "zain.master@example.com",
    "lukas.kawulok@example.com",
    "stephen.pack@example.com",
    "terry.stone@example.com",
    "radim.moravec@example.com",
    "petr.klen@example.com",
    "chuy@example.com",
    "nalin@example.com"
]

In [63]:
members_info = pd.DataFrame({'member': email_list, 'email': members_emails})
members_info

,member,email
0,Zain Master,zain.master@example.com
1,Lukas Kawulok,lukas.kawulok@example.com
2,Stephen Pack,stephen.pack@example.com
3,Terry Stone,terry.stone@example.com
4,Radim Moravec,radim.moravec@example.com
5,Petr Klen,petr.klen@example.com
6,Chuy,chuy@example.com
7,Nal,nalin@example.com


In [39]:
meeting_results_action_list = list(meeting_results['action calls'])

In [ ]:
# NER bot 2
prompt4 = """You are a smart and intelligent Named Entity Recognition (NER) system. You will take in a list of names and a meeting transcription and a list of calls to action. For each name, you will identify the Unique Call to Action directed towards them if applicable.
If a name does not have a task to do, list the name followed by 'There were no calls to action directed towards you.'
If a name has a task to do, list the name followed by the task or tasks on the same continuous line.
Your output should be a list of !every Unique name from the list, and their tasks or lack of tasks.
Ensure each Unique Name entry and their actions are one continuous line.
An Example Output List:

Name 1: (Task)
Name 2: (Task)
Name 3: (Task)
Name X: There were no calls to action directed towards you.
"""

# Instantiate chain
chain3 = client.chat.completions.create(
    model='gpt-4-turbo',
    messages=[{'role': 'system', 'content': prompt4},
              {'role': 'user', 'content': f"Names: {str(names_list)}\n Meeting: {transcribed_audio}\n Call to action list: {meeting_results_action_list}"}]
)

names_calls = chain3.choices[0].message.content
name_calls = names_calls.strip('\n \n')
print(names_calls)

Zain Master: Schedule another meeting to ensure all points are covered. Plan API performance testing and add it to the roadmap. Implement a formal process for booking QA resources. Evaluate the possibility of having waves for releases. Enhance communication regarding release timelines and updates. Propose and agree on new ideas for testing and implementation improvement. Assess how to streamline communication with customers during release waves. Develop better strategies for cross-team collaboration and testing. Define clear steps for addressing issues found during testing. Formalize the process for requesting and managing QA resources in advance. Establish gates for different waves of releases and improve DNS routing. Consider multiple gateways for releasing to different customer environments. Conduct deeper technical demos for internal stakeholders. Implement proactive, cross-functional testing practices. Streamline communication between solution center and core teams. Closely monito

In [40]:
import re

delimiters = [r": ", "\n"]
pattern = "|".join(map(re.escape, delimiters))

names_calls_list = re.split(pattern, names_calls)
names_calls_list

['Zain Master',
 'Schedule another meeting to ensure all points are covered. Plan API performance testing and add it to the roadmap. Implement a formal process for booking QA resources. Evaluate the possibility of having waves for releases. Enhance communication regarding release timelines and updates. Propose and agree on new ideas for testing and implementation improvement. Assess how to streamline communication with customers during release waves. Develop better strategies for cross-team collaboration and testing. Define clear steps for addressing issues found during testing. Formalize the process for requesting and managing QA resources in advance. Establish gates for different waves of releases and improve DNS routing. Consider multiple gateways for releasing to different customer environments. Conduct deeper technical demos for internal stakeholders. Implement proactive, cross-functional testing practices. Streamline communication between solution center and core teams. Closely m

In [41]:
meeting_actions = {}
i = 0
while i < len(names_calls_list) - 1:
    name = names_calls_list[i].strip()
    action = names_calls_list[i + 1].strip()
    meeting_actions[name] = action
    i += 2

meeting_actions = pd.DataFrame.from_dict([meeting_actions]).T.reset_index().rename(columns={'index': 'member', 0: 'action'})
meeting_actions

,member,action
0,Zain Master,Schedule another meeting to ensure all points ...
1,Peter Lafferman,There were no calls to action directed towards...
2,Lukas Kawulok,Schedule another meeting to ensure all points ...
3,Stephen Pack,Schedule another meeting to ensure all points ...
4,Holly,There were no calls to action directed towards...
5,Vijay,There were no calls to action directed towards...
6,Radim Moravec,Schedule another meeting to ensure all points ...
7,Terry Stone,Schedule another meeting to ensure all points ...
8,Peter Yu,Implement a formal process for booking QA reso...
9,Gloria,There were no calls to action directed towards...


In [42]:
meeting_actions_emails = pd.merge(meeting_actions, members_info, on='member', how='outer').dropna()
meeting_actions_emails

,member,action,email
0,Gloria,There were no calls to action directed towards...,gloria@example.com
1,Holly,There were no calls to action directed towards...,peter.lougherman@example.com
2,Lukas Kawulok,Schedule another meeting to ensure all points ...,stephen.pack@example.com
3,Nalin,There were no calls to action directed towards...,chuy@example.com
4,Peter Lafferman,There were no calls to action directed towards...,lukas.kawulok@example.com
5,Peter Yu,Implement a formal process for booking QA reso...,petr.klen@example.com
6,Radim Moravec,Schedule another meeting to ensure all points ...,vijay@example.com
7,Stephen Pack,Schedule another meeting to ensure all points ...,terry.stone@example.com
8,Terry Stone,Schedule another meeting to ensure all points ...,radim.moravec@example.com
9,Vijay,There were no calls to action directed towards...,chitan@example.com


In [43]:
str(meeting_results)

'{\'summary\': \'Summary:\\n\\nThe transcript from April 29, 2025, captures a critical meeting led by Zain Master, aimed at refining the product release process for a company that develops highly customized software solutions. The primary challenge discussed is the frequent need for product releases driven by compatibility issues between the core product functionalities and customer-specific customizations. This has resulted in dissatisfaction among both customers and internal teams.\\n\\n**Key Strategies and Discussions:**\\n\\n1. **Enhanced Collaboration:**\\n   Zain proposes improved collaboration between the solution center, which has deep insights into customer-specific implementations, and the core product team responsible for initial testing. He suggests that aligning additional testing with specific customer environments could mitigate recurring issues.\\n\\n2. **Release in Waves:**\\n   The concept of releasing updates in "waves" is debated. This strategy is intended to manage

In [50]:
# Email bot
prompt5 = """You are an intelligent Email formatting Assistant. You will take in a summary and calls to action from a meeting transcript, as well as a name, with their corresponding specific action and Email address.
Your task is to write this person an email. Fill in the To: field with their email, output the summary, followed by all calls to actions. Then the corresponding specific action(s) for the member the Email is written for. An example Email Format is as follows:

To: sarah.chen@example.com

From: znpmeetingassistant@gmail.com

Subject: Project Phoenix - Kickoff Meeting Summary

This email summarizes the kickoff meeting for Project Phoenix, held on October 26, 2023.

! Warning ! Assignments of tasks and Summaries generated by this Bot may be innacurate. Please be sure to follow up with Meeting Organizer / Team Lead

Meeting Summary:

The meeting began with an overview of Project Phoenix's objectives: to develop and launch a new customer relationship management (CRM) system within the next six months. We discussed the project's scope, which includes migrating existing customer data, integrating with our current marketing automation platform, and training our sales and support teams on the new system.

Key discussion points included:

- Data Migration Strategy: We reviewed the proposed approach for migrating customer data from the legacy system to the new CRM.

- Integration Requirements: We discussed the necessary integrations with the marketing automation platform, focusing on data synchronization and workflow automation.

- Training Plan: The training team outlined the plan for developing and delivering training materials to sales and support staff.

- Timeline and Milestones: We established the initial project timeline, identifying key milestones and deadlines. A detailed project schedule will be shared next week.

- Budget Allocation: Initial budget was presented.

Calls to Action:

- The following actions were assigned to ensure the project stays on track:

- Sarah Chen: Finalize the data migration plan and submit it for review by November 2nd.

- John Smith: Define the API specifications for the marketing automation platform integration by November 5th.

- Emily Brown: Develop the initial training modules outline by November 9th.

- David Lee: Circulate the detailed project schedule by October 28th.

- Michael Davis: Provide a detailed budget breakdown by November 2nd.


Specific Call to Action for Sarah Chen:

Sarah, your immediate action is to finalize the data migration plan, including the specific steps, resources required, and potential risks. Please submit this plan to project_team@example.com by November 2nd.

Thank you for your contributions to the meeting. Please let me know if you have any questions or require further clarification.

Best regards,

ZNP Meeting Assistant*

* ! Warning ! Assignments of tasks and Summaries generated by this Bot may be innacurate. Please be sure to follow up with Meeting Organizer / Team Lead



Here is the information:
"""

def generate_emails(meeting_results, meeting_actions_emails):
    emails = []
    for _, row in meeting_actions_emails.iterrows():
        member_name = row['member']
        member_email = row['email']
        specific_action = row['action']

        # Construct the user prompt with the specific information for the member
        user_prompt = f"""
        Here is the meeting summary:
        {meeting_results}

        Here are the calls to action for all members:
        {meeting_actions_emails.to_string(index=False)}

        Here is the specific call to action for {member_name}:
        {specific_action}
        """

        messages = [
            {'role': 'system', 'content': prompt5},
            {'role': 'user', 'content': user_prompt}
        ]

        try:
            chain = client.chat.completions.create(
                model=OPENAI_MODEL,
                messages=messages
            )
            email_content = chain.choices[0].message.content
            emails.append(email_content) 
            print(f"Generated email for {member_name} ({member_email})")
        except Exception as e:
            print(f"An error occurred while generating email for {member_name}: {e}")
            emails.append(f"Error: Could not generate email for {member_name}.")

    return emails


# Generate and store emails
generated_emails = generate_emails(meeting_results, meeting_actions_emails)

# Print the list of generated emails
print("\n--- Generated Emails ---")
for email in generated_emails:
    print(email)

Generated email for Gloria (gloria@example.com)
Generated email for Holly (peter.lougherman@example.com)
Generated email for Lukas Kawulok (stephen.pack@example.com)
Generated email for Nalin (chuy@example.com)
Generated email for Peter Lafferman (lukas.kawulok@example.com)
Generated email for Peter Yu (petr.klen@example.com)
Generated email for Radim Moravec (vijay@example.com)
Generated email for Stephen Pack (terry.stone@example.com)
Generated email for Terry Stone (radim.moravec@example.com)
Generated email for Vijay (chitan@example.com)
Generated email for Zain Master (zain.master@example.com)

--- Generated Emails ---
To: gloria@example.com

From: znpmeetingassistant@gmail.com

Subject: Refinement of Product Release Process - Meeting Summary and Action Items

This email summarizes the critical meeting held on April 29, 2025, focusing on refining the product release process for a company developing highly customized software solutions.

Meeting Summary:

The meeting highlighted th

In [45]:
len(generated_emails)

11

In [51]:
print(generated_emails[1])

To: peter.lougherman@example.com

From: znpmeetingassistant@gmail.com

Subject: Refinement of Product Release Process - Meeting Summary and Action Items

This email summarizes the critical meeting held on April 29, 2025, led by Zain Master to refine the product release process for highly customized software solutions.

Meeting Summary:

The primary challenge discussed was the need for frequent product releases due to compatibility issues between core product functionalities and customer-specific customizations. Key strategies and discussions focused on enhanced collaboration, releasing updates in "waves," technical and infrastructure challenges, resource management, communication during testing, inclusion of the solution center in demo meetings, documentation, accessibility, structured participation, API and integration changes, security and performance testing.

Calls to Action:

The following actions were assigned to ensure effective product release management:

- Schedule another me

In [52]:
email_bones = []
for email in generated_emails:
    lines = email.splitlines()
    email_bones.append(lines)

email_bones[0]

['To: gloria@example.com',
 '',
 'From: znpmeetingassistant@gmail.com',
 '',
 'Subject: Refinement of Product Release Process - Meeting Summary and Action Items',
 '',
 'This email summarizes the critical meeting held on April 29, 2025, focusing on refining the product release process for a company developing highly customized software solutions.',
 '',
 'Meeting Summary:',
 '',
 'The meeting highlighted the challenges faced by the organization due to frequent product releases driven by compatibility issues between core product functionalities and customer-specific customizations. Key discussions and strategies covered in the meeting included:',
 '',
 '1. Enhanced Collaboration: Improving collaboration between the solution center and core product team.',
 '2. Release in Waves: Considering the strategy of releasing updates in waves.',
 '3. Technical and Infrastructure Challenges: Addressing technical challenges in managing different release waves.',
 '4. Resource Management: Effective b

In [53]:
to_list = []
for email in email_bones:
    to_list.append(email[0].replace('To: ', ''))

sender_list = []
for email in email_bones:
    sender_list.append(email[2].replace('From: ', ''))

subject_list = []
for email in email_bones:
    subject_list.append(email[4].replace('Subject: ', ''))

message_text_list = []
for email in email_bones:
    message_text_list.append(str(email[6:]).replace("''", '\n').replace("',", '\n').replace("\n,", '').replace('\n', '\n\n').replace('",', '\n\n').replace('"', '').replace("'", "").replace('[', '\n').replace(']', '\n'))


'''to = email_bones[0]
cc = email_bones[2]
sender = email_bones[4]
subject = email_bones[6]
message_text = str(email_bones[8:]).replace("''", '\n').replace("',", '\n').replace("\n,", '').replace('\n', '\n\n').replace('",', '\n\n').replace('"', '').replace("'", "").replace('[', '\n').replace(']', '\n')'''


print(to_list[0])
print(sender_list[0])
print(subject_list[0])
print(message_text_list[0])


gloria@example.com
znpmeetingassistant@gmail.com
Refinement of Product Release Process - Meeting Summary and Action Items

This email summarizes the critical meeting held on April 29, 2025, focusing on refining the product release process for a company developing highly customized software solutions.

  Meeting Summary:

  The meeting highlighted the challenges faced by the organization due to frequent product releases driven by compatibility issues between core product functionalities and customer-specific customizations. Key discussions and strategies covered in the meeting included:

  1. Enhanced Collaboration: Improving collaboration between the solution center and core product team.

 2. Release in Waves: Considering the strategy of releasing updates in waves.

 3. Technical and Infrastructure Challenges: Addressing technical challenges in managing different release waves.

 4. Resource Management: Effective booking and allocation of QA resources.

 5. Communication During Testin

In [54]:
import base64
from email.message import EmailMessage

In [55]:
import os.path
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [56]:
SCOPES = ['https://www.googleapis.com/auth/gmail.send']


def create_message(sender, to, subject, message_text):
    message = EmailMessage()
    message.set_content(message_text)
    message['To'] = to
    message['From'] = sender
    message['Subject'] = subject
    return {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}


def send_message(service, user_id, message):
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                 .execute())
        print('Message Id: %s' % message['id'])
        return message
    except Exception as error:
        print(f'An error occurred: {error}')

def email_main(to_list, subject_list, message_text_list,):
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=60010)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    try:
        service = build('gmail', 'v1', credentials=creds)
        sender = "znpmeetingassistant@gmail.com"
        for to, subject, message_text in zip(to_list, subject_list, message_text_list):
            message = create_message(sender, to, subject, message_text)
            send_message(service, "me", message)
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
email_main(to_list, subject_list, message_text_list)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1092722805024-tu2g85irn4q8b7ap26lon70i902l0i73.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60010%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=Ce2VFgNHqulPIa1amxu3VUyDmPSQ9i&access_type=offline
